## Imports

In [1]:
from midi_to_dataframe import NoteMapper, MidiReader, MidiWriter
import IPython
from IPython.display import Image, IFrame
from PIL import Image
import seaborn as sns
import pandas as pd
import numpy as np
import os
import json
import music21
import pickle

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

2023-01-02 00:02:16.883913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 00:02:19.022605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/maarij/anaconda3/envs/tf/lib/
2023-01-02 00:02:19.023188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/maarij/anaconda3/envs/tf/lib/
2023-01-02 00:02:19.023221: W tensorflow/compiler/tf2ten

# Load Mappings File

A **NoteMapper** object encapsulates how MIDI note information is converted to text to be displayed within a DataFrame. This object is initialized from a JSON file, containing three objects:

* **midi-to-text**: JSON mapping of MIDI program numbers to their textual representation. Used when converting MIDI files to DataFrames.
    * For example: *{"0": "piano"}*
* **text-to-midi**: JSON mapping of textual representations of MIDI instruments to MIDI program numbers. Used when writing DataFrames to MIDI.
    * For example: *{"piano": 0}*
* **durations**: JSON mapping of textual representations of MIDI instruments to predefined quantization values (in quarter notes). Used when converted MIDI files to DataFrames.
    * For example: *{"piano": [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3, 4, 6, 8, 12, 16]}*

In [2]:
note_mapping_config_path = "./config/map-to-group.json"
note_mapper = NoteMapper(note_mapping_config_path)

## Convert a MIDI file to a DataFrame

The **MidiReader** object is used to read a MIDI file from disk and convert it to a **DataFrame**. A **NoteMapper** object is passed to the MidiReader upon initialization to handle the MIDI to text conversion of note durations and program names.

In [3]:
reader = MidiReader(note_mapper)

# getting midi files
filepath = "./datasets/dataset_pop/"
MidiDataDF = pd.DataFrame()
count = 0

for filename in os.listdir(filepath):
    if filename.endswith(".midi"):
        
        # create file path
        count += 1
        print(count, filename, end = " ")
        fullFilePath = filepath+filename

        # read file as dataframe
        tempDF = reader.convert_to_dataframe(fullFilePath)
        print(tempDF.shape[0])
        MidiDataDF = MidiDataDF.append(tempDF)


1 Firework.midi 1853
2 SomeoneForMe.midi 2416
3 MyHeartWillGoOn.midi 2375
4 Everytime.midi 1647
5 FixYou.midi 2625
6 Speechless.midi 1297
7 EnterSandman.midi 2720
8 HotNCold.midi 1873
9 CliffsOfDover.midi 1476
10 MissionImpossible.midi 1295
11 P.I.M.P..midi 1440
12 JustTheWayYouAre.midi 1593
13 FreshPrinceOfBelAir.midi 960
14 ThinkingOutLoud.midi 1149
15 Someonelikeyouwitlyric.midi 1322
16 YoureBeautiful.midi 1074
17 GangnamStyle.midi 1953
18 HungUp.midi 2031
19 GiveYourHeartABreak.midi 1664
20 Immortals.midi 1377
21 AnotherBrickInTheWall.midi 1509
22 EveryPlanetWeReachIsDead.midi 1680
23 PurpleRain.midi 1346
24 SmokeOnTheWater.midi 2627
25 WhatMakesYouBeautiful.midi 1664
26 LoveMeLikeYouDo.midi 1185
27 Midnight.midi 1943
28 YouMakeMeWanna.midi 1168
29 Hello.midi 1521
30 StayWithMe.midi 961
31 SweetHomeAlabama.midi 1442
32 TeAmo.midi 1171
33 Moveslikejagger.midi 1723
34 HarryPotter.midi 193
35 Simpsons.midi 991
36 Payphone.midi 1575
37 Royals.midi 1103
38 SweetDreams.midi 1680
39 Final

In [4]:
MidiDataDF

,timestamp,bpm,time_signature,measure,beat,notes
0,0,125.0,4/4,1,1.00,"piano_d#5_1.0,piano_c5_1.0,piano_g#4_1.0"
1,24,125.0,4/4,1,1.25,rest
2,48,125.0,4/4,1,1.50,rest
3,72,125.0,4/4,1,1.75,rest
4,96,125.0,4/4,1,2.00,"piano_g#4_1.0,piano_c5_1.0,piano_d#5_1.0"
...,...,...,...,...,...,...
1710,51300,120.0,4/4,107,4.50,rest
1711,51330,120.0,4/4,107,4.75,"bass_b2_0.25,bass_a#2_0.25,bass_a2_0.25"
1712,51360,120.0,4/4,108,1.00,"bass_g#2_0.25,bass_g2_0.25,bass_f#2_0.25,bass_..."
1713,51390,120.0,4/4,108,1.25,guitar_g#5_1.75


## MIDI DataFrame

The created DataFrame object contains the sequence of musical notes found in the input MIDI file, quantized by 16th notes and the following rows:

* **timestamp**: the MIDI timestamp (tick)
* **bpm**: the beats per minute at the timestamp
* **time_signature**: the time signature at the timestamp
* **measure**: the measure number at the timestamp
* **beat**: the downbeat within the current measure at the timestamp, in quarter notes
* **notes**: a textual representation of the notes played at the current timestamp

In [5]:
NotesDataDF = MidiDataDF[["notes"]]
NotesDataDF

,notes
0,"piano_d#5_1.0,piano_c5_1.0,piano_g#4_1.0"
1,rest
2,rest
3,rest
4,"piano_g#4_1.0,piano_c5_1.0,piano_d#5_1.0"
...,...
1710,rest
1711,"bass_b2_0.25,bass_a#2_0.25,bass_a2_0.25"
1712,"bass_g#2_0.25,bass_g2_0.25,bass_f#2_0.25,bass_..."
1713,guitar_g#5_1.75


## Vocabulary Building

In [11]:
vocabulary = ["rest"] # add rest by default

### Instruments

In [12]:
instruments = ['bass', 'synthlead', 'synthfx', 'reed',
               'percussive', 'organ', 'guitar', 'pipe',
               'soundfx', 'chromatic', 'ethnic', 'piano',
               'brass', 'synthpad', 'ensemble', 'strings']

percussionInstruments = ['acousticbassdrum', 'bassdrum', 'rimshot', 'acousticsnare',
                         'clap', 'snare', 'lowfloortom', 'closedhat', 'highfloortom',
                         'pedalhat', 'lowtom', 'openhat', 'lowmidtom', 'highmidtom',
                         'crashcymbal', 'hightom', 'ridecymbal', 'chinesecymbal',
                         'ridebell', 'tambourine', 'splashcymbal', 'cowbell', 'vibraslap',
                         'highbongo', 'lowbongo', 'mutehighconga', 'openhighconga', 'lowconga',
                         'hightimbale', 'lowtimbale', 'highagogo', 'lowagogo', 'cabasa',
                         'maracas', 'shortwhistle', 'longwhistle', 'shortguiro', 'longguiro',
                         'claves', 'highwoodblock', 'lowwoodblock', 'mutecuica', 'opencuica',
                         'mutetriangle', 'opentriangle']

### Chords

In [13]:
notesTemp = list(NotesDataDF["notes"])
chordsList = []

for i in notesTemp:
    if i != "rest":
        indexSplit = i.split(",")
        for j in indexSplit:
            chord = j.split("_")
            if chord[0] != "percussion":
                chordsList.append(chord[1])
        
chordsList = set(chordsList)
print(chordsList)

{'d#2', 'c#6', 'a#6', 'b1', 'b6', 'b0', 'f#5', 'b8', 'd5', 'g8', 'd8', 'e6', 'c5', 'c#8', 'g#2', 'c1', 'a5', 'c8', 'a6', 'g3', 'a#7', 'a#2', 'a2', 'c9', 'g7', 'c#4', 'a#8', 'a#3', 'c#7', 'd#9', 'e5', 'a#4', 'g#8', 'g9', 'd#1', 'd#6', 'f3', 'g1', 'b4', 'c6', 'd#7', 'c7', 'd1', 'a7', 'f#0', 'f#6', 'g#4', 'f7', 'd#0', 'a#0', 'f6', 'g4', 'a4', 'e4', 'd#3', 'c2', 'f2', 'd#5', 'b3', 'd7', 'd#8', 'g#1', 'f4', 'e9', 'g#3', 'c#0', 'b5', 'f5', 'c#2', 'e3', 'f#1', 'g0', 'd6', 'd0', 'f#4', 'c#1', 'f#2', 'e0', 'c#3', 'b7', 'f8', 'c3', 'd2', 'g2', 'f0', 'g#6', 'd9', 'g#7', 'e2', 'a3', 'f#7', 'c0', 'c#5', 'd#4', 'a#5', 'c#9', 'f#3', 'a8', 'e8', 'b2', 'g#5', 'f#8', 'e1', 'g6', 'e7', 'd3', 'c4', 'f9', 'g5', 'f1', 'd4', 'a1', 'a#1'}


### Durations

In [14]:
f = open(note_mapping_config_path)
jsonData = json.load(f)
f.close()

print(jsonData["durations"])

{'piano': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'chromatic': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'organ': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'guitar': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0, 24.0, 28.0, 32.0], 'bass': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'strings': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'ensemble': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'brass': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'reed': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'pipe': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'synthlead': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'synthpad': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16

### Build Vocabulary List

In [15]:
for i in instruments:
    for c in chordsList:
        for d in jsonData["durations"][i]:
            word = str(i) + "_" + str(c) + "_" + str(d)
            vocabulary.append(word)
            
for p in percussionInstruments:
    word = "percussion_" + str(p) + "_0.25"
    vocabulary.append(word)
            
print(len(vocabulary))
print(vocabulary)

21629
['rest', 'bass_d#2_0.25', 'bass_d#2_0.5', 'bass_d#2_0.75', 'bass_d#2_1.0', 'bass_d#2_1.25', 'bass_d#2_1.5', 'bass_d#2_1.75', 'bass_d#2_2.0', 'bass_d#2_2.5', 'bass_d#2_3.0', 'bass_d#2_4.0', 'bass_d#2_6.0', 'bass_d#2_8.0', 'bass_d#2_12.0', 'bass_d#2_16.0', 'bass_c#6_0.25', 'bass_c#6_0.5', 'bass_c#6_0.75', 'bass_c#6_1.0', 'bass_c#6_1.25', 'bass_c#6_1.5', 'bass_c#6_1.75', 'bass_c#6_2.0', 'bass_c#6_2.5', 'bass_c#6_3.0', 'bass_c#6_4.0', 'bass_c#6_6.0', 'bass_c#6_8.0', 'bass_c#6_12.0', 'bass_c#6_16.0', 'bass_a#6_0.25', 'bass_a#6_0.5', 'bass_a#6_0.75', 'bass_a#6_1.0', 'bass_a#6_1.25', 'bass_a#6_1.5', 'bass_a#6_1.75', 'bass_a#6_2.0', 'bass_a#6_2.5', 'bass_a#6_3.0', 'bass_a#6_4.0', 'bass_a#6_6.0', 'bass_a#6_8.0', 'bass_a#6_12.0', 'bass_a#6_16.0', 'bass_b1_0.25', 'bass_b1_0.5', 'bass_b1_0.75', 'bass_b1_1.0', 'bass_b1_1.25', 'bass_b1_1.5', 'bass_b1_1.75', 'bass_b1_2.0', 'bass_b1_2.5', 'bass_b1_3.0', 'bass_b1_4.0', 'bass_b1_6.0', 'bass_b1_8.0', 'bass_b1_12.0', 'bass_b1_16.0', 'bass_b6_0.25', 

### Create Dictionary to Map Word onto Integers

In [38]:
vocabMappings = dict(zip(vocabulary, range(0, len(vocabulary))))
print(vocabMappings)

{'rest': 0, 'bass_f#1_0.25': 1, 'bass_f#1_0.5': 2, 'bass_f#1_0.75': 3, 'bass_f#1_1.0': 4, 'bass_f#1_1.25': 5, 'bass_f#1_1.5': 6, 'bass_f#1_1.75': 7, 'bass_f#1_2.0': 8, 'bass_f#1_2.5': 9, 'bass_f#1_3.0': 10, 'bass_f#1_4.0': 11, 'bass_f#1_6.0': 12, 'bass_f#1_8.0': 13, 'bass_f#1_12.0': 14, 'bass_f#1_16.0': 15, 'bass_c9_0.25': 16, 'bass_c9_0.5': 17, 'bass_c9_0.75': 18, 'bass_c9_1.0': 19, 'bass_c9_1.25': 20, 'bass_c9_1.5': 21, 'bass_c9_1.75': 22, 'bass_c9_2.0': 23, 'bass_c9_2.5': 24, 'bass_c9_3.0': 25, 'bass_c9_4.0': 26, 'bass_c9_6.0': 27, 'bass_c9_8.0': 28, 'bass_c9_12.0': 29, 'bass_c9_16.0': 30, 'bass_g7_0.25': 31, 'bass_g7_0.5': 32, 'bass_g7_0.75': 33, 'bass_g7_1.0': 34, 'bass_g7_1.25': 35, 'bass_g7_1.5': 36, 'bass_g7_1.75': 37, 'bass_g7_2.0': 38, 'bass_g7_2.5': 39, 'bass_g7_3.0': 40, 'bass_g7_4.0': 41, 'bass_g7_6.0': 42, 'bass_g7_8.0': 43, 'bass_g7_12.0': 44, 'bass_g7_16.0': 45, 'bass_d#1_0.25': 46, 'bass_d#1_0.5': 47, 'bass_d#1_0.75': 48, 'bass_d#1_1.0': 49, 'bass_d#1_1.25': 50, 'bass_

### Mapping Data To Integers (Forward Mapping)

In [39]:
notesTemp = list(NotesDataDF["notes"])
mappedNotes = []

for i in notesTemp:
    indexSplit = i.split(",")
    for j in indexSplit:
        if len(indexSplit) > 1:
            mapping = int(vocabMappings[j]) * (-1)
        else:
            mapping = int(vocabMappings[j])
        mappedNotes.append(mapping)

print(len(mappedNotes))
mappedNotes

1183421


[-15021,
 -14946,
 -14091,
 0,
 0,
 0,
 -14091,
 -14946,
 -15021,
 0,
 0,
 0,
 -14091,
 -14946,
 -15021,
 0,
 0,
 0,
 -14091,
 -14946,
 -15021,
 0,
 0,
 0,
 -14181,
 -15516,
 -14121,
 0,
 0,
 0,
 -14121,
 -15516,
 -14181,
 0,
 0,
 0,
 -14121,
 -15516,
 -14181,
 0,
 0,
 0,
 -14121,
 -15516,
 -14181,
 0,
 0,
 0,
 -15501,
 -14091,
 -14946,
 0,
 0,
 0,
 -14946,
 -14091,
 -15501,
 0,
 0,
 0,
 -14946,
 -14091,
 -15501,
 0,
 0,
 0,
 -14946,
 -14091,
 -15501,
 0,
 0,
 0,
 -14391,
 -15501,
 -14091,
 0,
 0,
 0,
 -14091,
 -15501,
 -14391,
 0,
 0,
 0,
 -14091,
 -15501,
 -14391,
 0,
 0,
 0,
 -14091,
 -15501,
 -14391,
 0,
 0,
 0,
 -21591,
 -21585,
 -15021,
 -14946,
 -14091,
 0,
 21591,
 0,
 -21591,
 -21589,
 -15379,
 -14091,
 -14946,
 -15021,
 0,
 -21591,
 -21585,
 -15424,
 0,
 -21591,
 -15424,
 -14091,
 -14946,
 -15021,
 0,
 -21591,
 -21585,
 -15379,
 0,
 -21591,
 -21589,
 -15385,
 -14091,
 -14946,
 -15021,
 0,
 21591,
 0,
 -21591,
 -21585,
 -14181,
 -15516,
 -14121,
 0,
 21591,
 0,
 -21591,
 -2158

In [40]:
vocabularyChars = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"] # 10 is minus (-) and 11 is comma (,)
mappedNotesChars = []

for note in mappedNotes:
    temp = str(note)
    tempArr = [*temp]
    if tempArr[0] == "-":
        tempArr[0] = "10"
    tempArr.append("11")
    mappedNotesChars.extend(tempArr)
    
print(len(mappedNotesChars))
mappedNotesChars

7202917


['10',
 '1',
 '5',
 '0',
 '2',
 '1',
 '11',
 '10',
 '1',
 '4',
 '9',
 '4',
 '6',
 '11',
 '10',
 '1',
 '4',
 '0',
 '9',
 '1',
 '11',
 '0',
 '11',
 '0',
 '11',
 '0',
 '11',
 '10',
 '1',
 '4',
 '0',
 '9',
 '1',
 '11',
 '10',
 '1',
 '4',
 '9',
 '4',
 '6',
 '11',
 '10',
 '1',
 '5',
 '0',
 '2',
 '1',
 '11',
 '0',
 '11',
 '0',
 '11',
 '0',
 '11',
 '10',
 '1',
 '4',
 '0',
 '9',
 '1',
 '11',
 '10',
 '1',
 '4',
 '9',
 '4',
 '6',
 '11',
 '10',
 '1',
 '5',
 '0',
 '2',
 '1',
 '11',
 '0',
 '11',
 '0',
 '11',
 '0',
 '11',
 '10',
 '1',
 '4',
 '0',
 '9',
 '1',
 '11',
 '10',
 '1',
 '4',
 '9',
 '4',
 '6',
 '11',
 '10',
 '1',
 '5',
 '0',
 '2',
 '1',
 '11',
 '0',
 '11',
 '0',
 '11',
 '0',
 '11',
 '10',
 '1',
 '4',
 '1',
 '8',
 '1',
 '11',
 '10',
 '1',
 '5',
 '5',
 '1',
 '6',
 '11',
 '10',
 '1',
 '4',
 '1',
 '2',
 '1',
 '11',
 '0',
 '11',
 '0',
 '11',
 '0',
 '11',
 '10',
 '1',
 '4',
 '1',
 '2',
 '1',
 '11',
 '10',
 '1',
 '5',
 '5',
 '1',
 '6',
 '11',
 '10',
 '1',
 '4',
 '1',
 '8',
 '1',
 '11',
 '0',
 '11',


## RNN + LSTM Model

### Preparing Data For Input Into Model

In [41]:
class Data:
    features = []
    targets = []
    featureLength = 1000
    def __init__(self, features, targets, featureLength):
        self.features = features
        self.targets = targets
        self.featureLength = featureLength

In [42]:
print("To NP:")
mappedNotesChars = np.array(mappedNotesChars, dtype=float)

d = Data([],[],1000)

for i in range(len(mappedNotesChars)-d.featureLength):
    #print("Index:", i)
    tempF = mappedNotesChars[i:i+d.featureLength]
    d.features.append(tempF)
    tempT = mappedNotesChars[i+d.featureLength]
    d.targets.append(tempT)
    
n_patterns = len(d.targets)

To NP:


In [43]:
print(len(d.targets))
print(len(d.features[0]))

7201917
1000


In [45]:
print(d.features[0][0])
print(d.targets[0])

10.0
1.0


In [ ]:
pickle.dump(d, open('pop_data.pickle', 'wb'))

In [17]:
features = np.reshape(features, (n_patterns, featureLength, 1))

MemoryError: Unable to allocate 53.7 GiB for an array with shape (7206088, 1000) and data type float64

In [ ]:

targets = np.array(targets)
targets = to_categorical(targets, len(vocabularyChars))

In [ ]:
print(features[0], targets[1])

### Create and Train Model

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(features.shape[1],features.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(vocabulary), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(features, targets, epochs=20, batch_size=128, callbacks=callbacks_list)

### Mapping Integers to Data (Backward Mapping)

In [ ]:
output = mappedNotes[0:2000]
concatStr = ""
reverseMapping = []

for i in output:
    if i < 0:
        i = i * (-1)
        result = [new_k for new_k in vocabMappings.items() if new_k[1] == i][0][0]
        concatStr = concatStr + "," + result
    else:
        if concatStr != "":
            reverseMapping.append(concatStr.lstrip(","))
            concatStr = ""
        result = [new_k for new_k in vocabMappings.items() if new_k[1] == i][0][0]
        reverseMapping.append(result)
        
reverseMapping

## Convert a DataFrame to a MIDI File

The **MidiWriter** object handles writing properly-formatted **DataFrames** as playable MIDI files. A **NoteMapper** object is passed to the MidiWriter upon initialization to handle the text to MIDI conversion of note durations and program names. The path and filename of the output MIDI file is specified in the *convert_to_midi* call of the MidiWriter.

In [ ]:
outputDF = pd.DataFrame(reverseMapping, columns =['notes'])
outputDF["bpm"] = 125

cols = outputDF.columns.tolist()
cols = cols[-1:] + cols[:-1]
outputDF = outputDF[cols]

outputDF

In [ ]:
# Drop the first 15 rows of the dataframe, which represented 1 measure of silence

# Write the modified DataFrame to disk as a playable MIDI file
writer = MidiWriter(note_mapper)
writer.convert_to_midi(outputDF, "./output.midi")

parsed = music21.converter.parse("./output.midi")
parsed.write('musicxml.png', fp='./sheets/Score')
pdfPath = parsed.write('lily.pdf', fp='./sheets/Score')

filepath = "./sheets/"
for filename in os.listdir(filepath):
    if filename.endswith(".png"):
        im = Image.open(filepath+filename)
        bg = Image.new("RGB", im.size, (255,255,255))
        bg.paste(im,im)
        os.remove(filepath+filename)
        filename = filename.replace(".png",".jpg")
        bg.save(filepath+filename)

os.remove(filepath + "Score")
os.remove(filepath + "Score.musicxml")

IFrame(str(pdfPath), width=900, height=800)